# Predictive Maintenance
Predictive maintenance encompasses a variety of topics, including but not limited to: failure prediction, failure diagnosis (root cause analysis), failure detection, failure type classification, and recommendation of mitigation or maintenance actions after failure. As part of the Azure Machine Learning offering, Microsoft provides a template that helps data scientists easily build and deploy a predictive maintenance solution. This predictive maintenance template focuses on the techniques used to predict when an in-service machine will fail, so that maintenance can be planned in advance.

Three modeling solutions are provided in this template to accomplish the following tasks.
* Regression: Predict the Remaining Useful Life (RUL), or Time to Failure (TTF).
* Binary classification: Predict if an asset will fail within certain time frame (e.g. days). 
* Multi-class classification: Predict if an asset will fail in different time windows: E.g., fails in window [1, w0] days; fails in the window [w0+1,w1] days; not fail within w1 days 

The time units mentioned above can be replaced by working hours, cycles, mileage, transactions, etc. based on the actual scenario. 

In this notebool, we shall implement the Binary classification solution: Predict if an asset will fail within certain time frame (e.g. days). 

In [1]:
import keras

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# setting seed for reproducability
np.random.seed(1234)
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline

# Data Ingestion

In the following section, we ingest the training, test and ground truth datasets. The training data consists of multiple multivariate time series with "cycle" as the time unit, together with 21 sensor readings for each cycle. Each time series can be assumed as being generated from a different engine of the same type. The testing data has the same data schema as the training data. The only difference is that the data does not indicate when the failure occurs. Finally, the ground truth data provides the number of remaining working cycles for the engines in the testing data. 

The data is the example of simulated aircraft engine run-to-failure events to demonstrate the predictive maintenance modeling process. The implicit assumption of modeling data as done below is that the asset of interest has a progressing degradation pattern, which is reflected in the asset's sensor measurements. By examining the asset's sensor values over time, the machine learning algorithm can learn the relationship between the sensor values and changes in sensor values to the historical failures in order to predict failures in the future.


Datasets were be downloaded from Azure blob and stored in sub-folder "data"
* Training data: It is the aircraft engine run-to-failure data. http://azuremlsamples.azureml.net/templatedata/PM_train.txt
* Testing data: It is the aircraft engine operating data without failure events recorded. http://azuremlsamples.azureml.net/templatedata/PM_test.txt
* Ground truth data: It contains the information of true remaining cycles for each engine in the testing data. http://azuremlsamples.azureml.net/templatedata/PM_truth.txt

The data schema for the training and testing data is described as below:
* id: Integer, aircraft engine identifier, range [1,100]
* cycle: Integer, time, in cycles
* setting1: Double, Operational setting 1
* setting2: Double, Operational setting 2
* setting3: Double, Operational setting 3
* s1: Double, Sensor measurement 1
* s2: Double, Sensor measurement 2
....
* s21: Double, Sensor measurement 21

Total: 26 columns

The training data ("PM_train.txt") consists of multiple multivariate time series with "cycle" as the time unit, together with 21 sensor readings for each cycle. Each time series can be assumed as being generated from a different engine of the same type. Each engine is assumed to start with different degrees of initial wear and manufacturing variation, and this information is unknown to the user. In this simulated data, the engine is assumed to be operating normally at the start of each time series. It starts to degrade at some point during the series of the operating cycles. The degradation progresses and grows in magnitude. When a predefined threshold is reached, then the engine is considered unsafe for further operation. In other words, the last cycle in each time series can be considered as the failure point of the corresponding engine. Taking the sample training data shown in the following table as an example, the engine with id=1 fails at cycle 192, and engine with id=2 fails at cycle 287. 

The testing data ("PM_test.txt") has the same data schema as the training data. The only difference is that the data does not indicate when the failure occurs (in other words, the last time period does NOT represent the failure point). Taking the sample testing data shown in the following table as an example, the engine with id=1 runs from cycle 1 through cycle 31. It is not shown how many more cycles this engine can last before it fails.

The ground truth data ("PM_truth.txt") provides the number of remaining working cycles for the engines in the testing data. Taking the sample ground truth data shown in the following table as an example, the engine with id=1 in the testing data can run another 112 cycles before it fails. 

In [3]:
# read training data
train_df = pd.read_csv("data\\PM_train.txt", sep=" ", header=None)
# drop 02 last columns that are not used
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
# specify column names
train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']

In [4]:
# sort train_data
train_df = train_df.sort_values(['id', 'cycle'])
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [5]:
# read test data
test_df = pd.read_csv("data\\PM_test.txt", sep=" ", header=None)
# drop 02 last columns that are not used
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
# specify column names
test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']

In [6]:
test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


In [7]:
# read ground truth data
truth_df = pd.read_csv("data\\PM_truth.txt", sep=" ", header=None)
# remove the last column that not necessary
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

In [8]:
# set column name 'more' for remaining working cycles
truth_df.columns = ['more']
# add column for engine id
truth_df['id'] = truth_df.index + 1
truth_df.head()

,more,id
0,112,1
1,98,2
2,69,3
3,82,4
4,91,5


# Data Labelling

First step is to generate labels for the training data which are Remaining Useful Life (RUL). Here, we will only make use of "label1" for binary clasification, while trying to answer the question: is a specific engine going to fail within w1 cycles?

In [9]:
# Dara labelling - Generate column RUL (Remain Useful Lifetime)
# get the max number of cycles of each engine. It is when the engine is not safe anymore
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
# rename column names
rul.columns = ['id', 'max']
# merge with training dataframe
train_df = train_df.merge(rul, on=['id'], how='left')
# calculate RUL
train_df['RUL'] = train_df['max'] - train_df['cycle']
# column max is no longer necessary, remove it
train_df.drop('max', axis=1, inplace=True)
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [10]:
# generate labels for training data
w1 = 30 # time window 1 = 30 cycles
w0 = 15 # time window 2 = 15 cycles

# label1: if RUL is less than time window 1
train_df['label1'] = np.where(train_df['RUL'] < w1, 1, 0)
# label2 = 2 is RUL is less than time window 0
train_df['label2'] = train_df['label1']
train_df.loc[train_df['RUL'] < w0, 'label2'] = 2


In [11]:
train_df[train_df['label1']==1].head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,RUL,label1,label2
162,1,163,0.0003,-0.0004,100.0,518.67,642.85,1600.54,1421.09,14.62,...,8.5129,0.03,393,2388,100.0,38.65,23.1419,29,1,1
163,1,164,0.0005,-0.0002,100.0,518.67,643.17,1598.96,1416.76,14.62,...,8.4803,0.03,394,2388,100.0,38.62,23.1761,28,1,1
164,1,165,0.0010,0.0004,100.0,518.67,642.76,1597.03,1408.09,14.62,...,8.4922,0.03,393,2388,100.0,38.59,23.2129,27,1,1
165,1,166,-0.0022,-0.0003,100.0,518.67,643.34,1596.72,1422.37,14.62,...,8.4663,0.03,395,2388,100.0,38.62,23.1450,26,1,1
166,1,167,0.0012,0.0003,100.0,518.67,643.02,1593.83,1414.72,14.62,...,8.4632,0.03,394,2388,100.0,38.77,23.3011,25,1,1


In [12]:
train_df[train_df['label2']==2].head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,RUL,label1,label2
177,1,178,-0.0001,0.0002,100.0,518.67,643.38,1605.33,1424.65,14.62,...,8.4770,0.03,395,2388,100.0,38.65,23.1770,14,1,2
178,1,179,0.0023,-0.0003,100.0,518.67,642.86,1592.56,1429.45,14.62,...,8.4947,0.03,395,2388,100.0,38.77,23.1888,13,1,2
179,1,180,-0.0024,0.0000,100.0,518.67,643.58,1599.87,1417.14,14.62,...,8.4737,0.03,395,2388,100.0,38.86,23.0070,12,1,2
180,1,181,-0.0006,0.0005,100.0,518.67,643.44,1596.71,1420.64,14.62,...,8.4720,0.03,397,2388,100.0,38.34,23.1652,11,1,2
181,1,182,-0.0010,0.0003,100.0,518.67,644.21,1602.08,1426.62,14.62,...,8.5158,0.03,397,2388,100.0,38.43,23.1787,10,1,2


# Feature Engineering

In [13]:
# colume "cycle" could be used as a feature, copy it, to normalize it later
train_df['cycle_norm'] = train_df['cycle']

#MinMax Normalization
cols_normalize = train_df.columns.difference(['id', 'cycle', 'RUL', 'label1', 'label2'])
minmax_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(minmax_scaler.fit_transform(train_df[cols_normalize]),
                             columns = cols_normalize,
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,RUL,label1,label2,cycle_norm
0,1,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,0,0,0.00000
1,1,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,0,0,0.00277
2,1,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,0,0,0.00554
3,1,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,0,0,0.00831
4,1,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,0,0,0.01108


In [14]:
# Now, normalize the test data using the same minmax scaler

# keep the "cycle" column of test data
test_df['cycle_norm'] = test_df['cycle']
norm_test_df = pd.DataFrame(minmax_scaler.transform(test_df[cols_normalize]),
                           columns = cols_normalize,
                           index = test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)
test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,cycle_norm
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.220588,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.00000
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.264706,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.00277
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.220588,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.00554
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.250000,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.00831
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.220588,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.01108


In [15]:
# generate column max for test data
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id','max']
# calculate truth's max working cycle
truth_df['max'] = rul['max'] + truth_df['more']
# "more" column is no more needed
truth_df.drop('more', axis=1, inplace=True)
truth_df.head()

,id,max
0,1,143
1,2,147
2,3,195
3,4,188
4,5,189


In [16]:
# generate column RUL for test data
test_df = test_df.merge(truth_df, on='id', how='left')
test_df['RUL'] = test_df['max'] - test_df['cycle']
# "max" column is no more necessary
test_df.drop('max', axis=1, inplace=True)
test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,cycle_norm,RUL
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.00000,142
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.00277,141
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.00554,140
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.00831,139
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.01108,138


In [17]:
# generate label columns w1 & w0 for test data
test_df['label1'] = np.where(test_df['RUL'] < w1, 1, 0)
test_df['label2'] = test_df['label1']
test_df.loc[test_df['RUL'] < 0, 'label2'] = 2
test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.00000,142,0,0
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.00277,141,0,0
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.00554,140,0,0
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.00831,139,0,0
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.01108,138,0,0


# Modelling
The traditional predictive maintenance machine learning models are based on feature engineering which is manual construction of right features using domain expertise and similar methods. This usually makes these models hard to reuse since feature engineering is specific to the problem scenario and the available data which varies from one business to the other. Perhaps the most attractive part of applying deep learning in the predictive maintenance domain is the fact that these networks can automatically extract the right features from the data, eliminating the need for manual feature engineering.

When using LSTMs in the time-series domain, one important parameter to pick is the sequence length which is the window for LSTMs to look back. This may be viewed as similar to picking window_size = 5 cycles for calculating the rolling features in the Predictive Maintenance Template which are rolling mean and rolling standard deviation for 21 sensor values. The idea of using LSTMs is to let the model extract abstract features out of the sequence of sensor values in the window rather than engineering those manually. The expectation is that if there is a pattern in these sensor values within the window prior to failure, the pattern should be encoded by the LSTM.

One critical advantage of LSTMs is their ability to remember from long-term sequences (window sizes) which is hard to achieve by traditional feature engineering. For example, computing rolling averages over a window size of 50 cycles may lead to loss of information due to smoothing and abstracting of values over such a long period, instead, using all 50 values as input may provide better results. While feature engineering over large window sizes may not make sense, LSTMs are able to use larger window sizes and use all the information in the window as input. Below, we illustrate the approach.

In [18]:
# set-up the window size
sequence_length = 50

Keras LSTM layers expect an input in the shape of a numpy array of 3 dimensions (samples, time steps, features) where samples is the number of training sequences, time steps is the look back window or sequence length and features is the number of features of each sequence at each time step.

In [19]:
# create a function to reshape features into (samples, time steps, features)
def gen_sequence(id_df, seq_length, seq_cols):
    # Only sequences that meet the window-length are considered, no padding is used.
    # This means for testing, we need to drop those which are below the window-length.
    # An alternative would be to pad sequences so that we can use shorter ones 
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements - seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

In [20]:
# pick the feature columns
sequence_cols = ['setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 'cycle_norm']

In [21]:
# generator for sequences
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) for id in train_df['id'].unique())

In [22]:
# generate sequences and convert to numpy arrays
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

(15631, 50, 25)

In [23]:
# create a function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

In [24]:
# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['label1']) for id in train_df['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

(15631, 1)

# LSTM Network

Next, we build a deep network. The first layer is an LSTM layer with 100 units followed by another LSTM layer with 50 units. Dropout is also applied after each LSTM layer to control overfitting. Final layer is a Dense output layer with single unit and sigmoid activation since this is a binary classification problem.

In [25]:
# build the network
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()

model.add(LSTM(
    input_shape=(sequence_length, nb_features),
    units=100,
    return_sequences = True))

model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=False))

model.add(Dropout(0.2))

model.add(Dense(units = nb_out, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 100)           50400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 80,651
Trainable params: 80,651
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
%%time
# fit the network
model.fit(seq_array,
          label_array,
          epochs=10,
          batch_size=200,
          validation_split = 0.05,
          verbose=1, 
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                                     min_delta=0,
                                                     patience=0,
                                                     verbose=0,
                                                     mode='auto')])

Train on 14849 samples, validate on 782 samples
Epoch 1/10
14849/14849 [==============================] - 30s - loss: 0.2411 - acc: 0.9007 - val_loss: 0.1554 - val_acc: 0.9412
Epoch 2/10
14849/14849 [==============================] - 18s - loss: 0.0931 - acc: 0.9624 - val_loss: 0.0970 - val_acc: 0.9501
Epoch 3/10
14849/14849 [==============================] - 18s - loss: 0.0874 - acc: 0.9644 - val_loss: 0.0413 - val_acc: 0.9885
Epoch 4/10
14849/14849 [==============================] - 18s - loss: 0.0648 - acc: 0.9731 - val_loss: 0.0332 - val_acc: 0.9923
Epoch 5/10
14849/14849 [==============================] - 18s - loss: 0.0669 - acc: 0.9706 - val_loss: 0.1033 - val_acc: 0.9565
Wall time: 1min 47s


In [28]:
# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Accuracy : {}'.format(scores[1]))

15631/15631 [==============================] - 6s     
Accuracy : 0.9534898370182161


In [29]:
# make predictions and compute confusion matrix
y_pred = model.predict_classes(seq_array, verbose=1, batch_size=200)
y_true = label_array
print('Confusion matrix\n- x-axis is true labels\n- y-axis is predicted labels')
cm = confusion_matrix(y_true, y_pred)
cm

15631/15631 [==============================] - 6s     
Confusion matrix
- x-axis is true labels
- y-axis is predicted labels


array([[11996,   635],
       [   92,  2908]], dtype=int64)

In [30]:
# compute precision and recall
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print('Precision = ', precision, '\n', 'recall = ', recall)

Precision =  0.820773355913 
 recall =  0.969333333333


Next, we look at the performance on the test data. In the Predictive Maintenance Template Step 1 of 3, only the last cycle data for each engine id in the test data is kept for testing purposes. In order to compare the results to the template, we pick the last sequence for each id in the test data.

In [31]:
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:]
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length ]

# convert to numpy array
seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
seq_array_test_last.shape

(93, 50, 25)

In [33]:
# labels in corresponding to above test sequences
y_mask = [len(test_df[test_df['id']==id]) >= sequence_length for id in test_df['id'].unique()]
label_array_test_last = test_df.groupby('id')['label1'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0], 1).astype(np.float32)
label_array_test_last.shape

(93, 1)

In [34]:
# test metrics
scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose=2)
print('Accuracy : {}'.format(scores_test[1]))

Accuracy : 0.9677419297156795


In [35]:
# make predictions and compute confusion matrix
y_pred_test = model.predict_classes(seq_array_test_last)
y_true_test = label_array_test_last
print('Confusion matrix\n- x-axis is true labels\n- y-axis is predicted labels')
cm = confusion_matrix(y_true_test, y_pred_test)
cm

64/93 [===================>..........] - ETA: 0sConfusion matrix
- x-axis is true labels
- y-axis is predicted labels


array([[65,  3],
       [ 0, 25]], dtype=int64)

In [37]:
# compute precision and recall
precision_test = precision_score(y_true_test, y_pred_test)
recall_test = recall_score(y_true_test, y_pred_test)
f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
print('Precision = ', precision_test, '\n', 'recall = ', recall_test, '\n', 'F1 score = ', f1_test)

Precision =  0.892857142857 
 recall =  1.0 
 F1 score =  0.943396226415


# Save the scaler & model into binary files

In [38]:
import pickle

In [40]:
# Save the scaler into a pickle file
scaler_pkl_path = "deploy\\scaler.pkl"
scaler_pkl = open(scaler_pkl_path, "wb")
pickle.dump(minmax_scaler, scaler_pkl)
scaler_pkl.close()

In [43]:
# Save the model into a hdf5 file
model_hdf5_path = "deploy\\model_predictve_maintenance_LSTM_30days_classifier.h5"
model.save(model_hdf5_path)

In [46]:
# Save the column normalizer index into a pickle file
cols_normalize_pkl_path = "deploy\\cols_normalize.pkl"
cols_normalize_pkl = open(cols_normalize_pkl_path, 'wb')
pickle.dump(cols_normalize, cols_normalize_pkl)
cols_normalize_pkl.close()